In [ ]:
from dotenv import load_dotenv
load_dotenv()


c:\Users\weraz\Documents\sem7\the-cleora\notebooks


In [8]:
from src.utils.data import load_data
from src.pipeline import create_pipeline
from src.Preprocessing import Preprocessing
from src.CleoraFacade import CleoraFacade
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from pathlib import Path

ModuleNotFoundError: No module named 'src'

In [24]:
import pandas as pd
headers = ['id']
for i in range(42):
    headers.append(i)
df = pd.read_csv("data/features.csv", names=headers, header=0)
print(df)

          id     0     1     2     3     4     5     6     7     8  ...  32  \
0          0  1574  3773  3571  2672  2478  2534  3129  3077  1171  ...   0   
1          1  1193   376    73   290  3129  1852  3077  1171  1022  ...   0   
2          2  1574  3773   925  1728  2815  2963  3077   364  1171  ...   0   
3          3  3964  3773  4003   928  1852  3077   364  1022  3763  ...   0   
4          4  1929  3773  1793  3511  1290  3129  3077   364  1171  ...   0   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...  ..   
37695  37695  1574  3773    73  1995  3554  1233  1789   345  3129  ...   0   
37696  37696  1929  3773  1663  1404   508   819  1852  3077   364  ...   0   
37697  37697  3433  3773  3104  1245  3129  1852  3077   364  1171  ...   0   
37698  37698  3730  3773  1695  2092  2954  1852  3077   364  1171  ...   0   
37699  37699  3433   509  1663  3437   676  3129  1852  3077  3763  ...   0   

       33  34  35  36  37  38  39  40  41  
0      

In [33]:
preprocessing = Preprocessing()
preprocessing.make_preprocessed_edges_file()
cleora = CleoraFacade(iterations=5)
cleora.run_cleora(Path.cwd() / "data" / "preprocessed_edges.txt")

In [35]:
train = preprocessing.train
test = preprocessing.test
print(train.head())

          id       name  ml_target
4773    4773      Nsima          0
36287  36287  jaycech3n          0
24241  24241  davendw49          1
27463  27463   mccordgh          0
6815    6815    oalders          0


In [36]:
embeddings_path = Path.cwd() / "data" / "embeddings" / "emb__cluster_id__node.out"
embeddings, dimension = cleora.load_embeddings(embeddings_path)

In [37]:
train = train.merge(embeddings, left_on='id', right_on='node', how='left').drop(columns=['node'])
test = test.merge(embeddings, left_on='id', right_on='node', how='left').drop(columns=['node'])
print(train.head())

      id       name  ml_target  \
0   4773      Nsima          0   
1  36287  jaycech3n          0   
2  24241  davendw49          1   
3  27463   mccordgh          0   
4   6815    oalders          0   

                                           embedding  
0  [0.026979137, -0.06968384, 0.08356073, 0.02421...  
1  [-0.029177433, -0.06258977, -0.035947073, -0.0...  
2  [0.08326291, -0.011103394, -0.06109125, 0.0916...  
3  [-0.049261425, -0.05946914, 0.033284802, -0.04...  
4  [-0.021889813, -0.1070307, 0.01412795, -0.0497...  


In [39]:
train = train.merge(df.iloc[:,0:10], on='id')
test = test.merge(df.iloc[:,0:10], on='id')
print(train.head())

      id       name  ml_target  \
0   4773      Nsima          0   
1  36287  jaycech3n          0   
2  24241  davendw49          1   
3  27463   mccordgh          0   
4   6815    oalders          0   

                                           embedding     0     1     2     3  \
0  [0.026979137, -0.06968384, 0.08356073, 0.02421...  3730  1330   277  1542   
1  [-0.029177433, -0.06258977, -0.035947073, -0.0...  1929  1330  1793   101   
2  [0.08326291, -0.011103394, -0.06109125, 0.0916...  3730  1033  1793  2771   
3  [-0.049261425, -0.05946914, 0.033284802, -0.04...  3730   509  3571  1559   
4  [-0.021889813, -0.1070307, 0.01412795, -0.0497...  2048  2325  3214  3129   

      4     5     6     7     8  
0  3129  3077   364  1022  3763  
1  2548  1290  3077   364  1171  
2  2947  3129  3077  1171  1022  
3  3596  3129   364  1022  3763  
4  1852  3077   364  1171  1022  


In [40]:
X_test = test.drop(columns=["id", "name", "ml_target"])
X_train = train.drop(columns=["id", "name", "ml_target"])
y_train = train['ml_target']
y_test = test['ml_target']
X_train.head()

,embedding,0,1,2,3,4,5,6,7,8
0,"[0.026979137, -0.06968384, 0.08356073, 0.02421...",3730,1330,277,1542,3129,3077,364,1022,3763
1,"[-0.029177433, -0.06258977, -0.035947073, -0.0...",1929,1330,1793,101,2548,1290,3077,364,1171
2,"[0.08326291, -0.011103394, -0.06109125, 0.0916...",3730,1033,1793,2771,2947,3129,3077,1171,1022
3,"[-0.049261425, -0.05946914, 0.033284802, -0.04...",3730,509,3571,1559,3596,3129,364,1022,3763
4,"[-0.021889813, -0.1070307, 0.01412795, -0.0497...",2048,2325,3214,3129,1852,3077,364,1171,1022


In [41]:
X_test = []
for i in test.values:
    l = i[3].tolist()
    for x in range(4, len(i)):
        l.append(i[x])
    X_test.append(l)
print(X_test[0])

X_train = []
for i in train.values:
    l = i[3].tolist()
    for x in range(4, len(i)):
        l.append(i[x])
    X_train.append(l)
print(X_train[0])

[-0.07880121, -0.08921524, -0.09372365, -0.11417484, 0.03372334, 0.044510406, -0.009739061, 0.015329454, 0.014584135, -0.0413674, -0.07673359, 0.16213013, 0.030030074, 0.098777734, 0.14929837, 0.07676522, 0.13830703, -0.045389593, -0.03793476, 0.27602834, -0.010514128, -0.069654055, -0.06446309, -0.066418566, 0.008946054, 0.09072224, -0.0060333973, 0.02252851, 0.04819063, -0.015175823, -0.037488803, 0.011543756, 0.051636368, 0.053291466, -0.15434681, -0.008824627, 0.0806972, -0.12225716, -0.042166088, -0.04048852, -0.015553097, -0.071669064, -0.050960477, 0.021979755, -0.044208672, -0.018501569, -0.10709141, 0.08408238, -0.044861022, 0.038537905, -0.04692758, -0.0039482242, -0.09939028, -0.09972539, 0.04268243, -0.09526344, -0.057105023, -0.035252757, -0.04012046, 0.004512539, 0.050589792, -0.0060870443, -0.03862141, -0.013215922, -0.092177525, 0.2018826, -0.15300916, -0.034758747, 0.08133207, -0.039115965, 0.055834997, 0.012077628, 0.01739094, -0.019238692, -0.13479745, 0.061656564, 0

In [42]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


In [23]:
models = {
    "KNeighbors": KNeighborsClassifier(n_neighbors=7), 
    "DecisionTree": DecisionTreeClassifier() 
}

for model_name, model in models.items():
        print(f"\nTraining pipeline with {model_name}...")
        pipeline = create_pipeline(model)
        
        pipeline.fit(X_train_scaled, y_train)
        y_pred = pipeline.predict(X_test_scaled)
        
        # evaluate model performance
        print(f"\n{model_name} Model Evaluation:")
        print(f"Accuracy: {pipeline.score(X_test_scaled, y_test):.4f}")
        print(classification_report(y_test, y_pred))


Training pipeline with KNeighbors...
[Pipeline] . (step 1 of 2) Processing cosine_similarity, total=  31.2s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   9.0s


c:\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")



KNeighbors Model Evaluation:
Accuracy: 0.7788
              precision    recall  f1-score   support

           0       0.81      0.92      0.86      5620
           1       0.61      0.36      0.45      1920

    accuracy                           0.78      7540
   macro avg       0.71      0.64      0.66      7540
weighted avg       0.76      0.78      0.76      7540


Training pipeline with DecisionTree...
[Pipeline] ........ (step 1 of 1) Processing classifier, total=  10.4s

DecisionTree Model Evaluation:
Accuracy: 0.7460
              precision    recall  f1-score   support

           0       0.84      0.81      0.83      5620
           1       0.50      0.54      0.52      1920

    accuracy                           0.75      7540
   macro avg       0.67      0.68      0.67      7540
weighted avg       0.75      0.75      0.75      7540



In [ ]:
models = {
    "KNeighbors": KNeighborsClassifier(n_neighbors=7), 
    "DecisionTree": DecisionTreeClassifier() 
}

for model_name, model in models.items():
        print(f"\nTraining pipeline with {model_name}...")
        pipeline = create_pipeline(model)
        
        pipeline.fit(X_train_scaled, y_train)
        y_pred = pipeline.predict(X_test_scaled)
        
        # evaluate model performance
        print(f"\n{model_name} Model Evaluation:")
        print(f"Accuracy: {pipeline.score(X_test_scaled, y_test):.4f}")
        print(classification_report(y_test, y_pred))


Training pipeline with KNeighbors...
[Pipeline] . (step 1 of 2) Processing cosine_similarity, total=  14.3s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   7.7s

KNeighbors Model Evaluation:
